## Requirements install

In [1]:
%pip install pandas
%pip install "betterproto[compiler]"
%pip install matplotlib
%pip install scipy
%pip install pyQt5
%pip install scikit-learn
%pip install numpy
%matplotlib qt      
%matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

## Pyproto generation

In [2]:
from utils.auto_generate_proto import generate_proto_classes

generate_proto_classes()

CMD: C:\ProgramData\chocolatey\bin\protoc.EXE --plugin=protoc-gen-python_betterproto=C:\Users\victo\Documents\in1144-data-mining-2025-2\.protoc_plugins\protoc-gen-python_betterproto.cmd -I C:\Users\victo\Documents\in1144-data-mining-2025-2\libs\protobufs\include\protobufs\pb\proto --python_betterproto_out=C:\Users\victo\Documents\in1144-data-mining-2025-2\proto\generated rc_log.proto
returncode: 0
STDOUT:
 
STDERR:
 Writing RCLog.py
Writing RoboCupSSL.py
Writing __init__.py

✅ Sucesso!


## Base includes 

In [3]:
from data_select.data_filter_operator import *
import pandas as pd
import matplotlib.pyplot as plt
from utils.utils import OUTPUT_DIR, data_frame_to_csv, LogFields, gel_2d_length_in_column
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
import scipy.stats as stats
import numpy as np
import math

## Path to log file

In [6]:
PREFIX_PATH = 'logs/'
LOG_FILE = 'group_phase_tigers_robocin.log.gz'

## Load log data

In [7]:
selects = [LogFields.PROCESSED_FRAME, LogFields.RAW_FRAME, LogFields.TELEMETRY, LogFields.ROBOTS_COMMAND, LogFields.REFEREE]

data_list = load_select_modules(PREFIX_PATH+LOG_FILE, selects)

# Data analysis

## Kick Data Split 

### 🧩 Descrição dos DataFrames

Durante a extração dos **eventos de chute (kick events)**, vários *dataframes* são gerados e filtrados para o mesmo intervalo temporal.  
Cada um representa um conjunto de dados diferente — bola, robôs, comandos e telemetria — obtidos de diferentes partes do sistema.

---

#### ⚽ `ball`
Contém informações processadas da bola ao longo do tempo, derivadas do sistema de visão.

| Coluna | Descrição |
|:--|:--|
| `timestamp` | Momento em que o frame foi publicado. |
| `position_x`, `position_y` | Posição da bola no campo, em milímetros. |
| `velocity_x`, `velocity_y` | Componentes da velocidade da bola nos eixos X e Y. |
| `acceleration_x`, `acceleration_y` | Componentes da aceleração da bola nos eixos X e Y. |
| `velocity_norm` | Módulo da velocidade. |
| `acceleration_norm` | Módulo da aceleração. |

Esses dados são usados para **detectar e analisar os chutes**, pois mostram mudanças bruscas de aceleração e velocidade.

---

#### 🤖 `processed_robots`
Contém a posição e velocidade **estimadas** de todos os robôs (aliados e adversários), calculadas a partir dos frames processados pelo sistema de visão.

| Coluna | Descrição |
|:--|:--|
| `timestamp` | Momento em que o frame foi salvo. |
| `team` | Indica se o robô pertence ao time aliado (`allies`) ou adversário (`enemies`). |
| `id` | Identificador do robô. |
| `position_x`, `position_y`, `position_w` | Posição e orientação do robô no campo. |
| `velocity_x`, `velocity_y` | Componentes da velocidade linear. |
| `velocity_norm` | Módulo da velocidade. |

---

#### 🧭 `commands`
Contém os comandos de movimentação e atuação enviados aos robôs aliados.

| Coluna | Descrição |
|:--|:--|
| `timestamp` | Momento em que o comando foi enviado. |
| `id` | Identificador do robô aliado. |
| `move_x`, `move_y`, `move_w` | Componentes de comando de movimento (velocidades desejadas nos eixos X, Y e angular). |
| `actuation_kick_strength` | Tempo de descarga dos capacitores. |
| `actuation_chip` | Indica se o chute foi do tipo *chip*. |
| `actuation_front` | Indica se o chute foi do tipo *front*. |
| `actuation_charge` | Indica que deve carregar os capacitores. |
| `actuation_dribbler` | Estado do dribbler (ligado/desligado). |
| `actuation_dribbler_velocity` | Velocidade do dribbler (RPM). |

---

#### 🔧 `telemetry`
Contém dados de telemetria enviados pelos robôs aliados.

| Coluna | Descrição |
|:--|:--|
| `timestamp` | Instante de recepção da telemetria. |
| `id` | Identificador do robô aliado. |
| `wheel1`, `wheel2`, `wheel3`, `wheel4` | Velocidade de cada roda individual. |
| `position_x`, `position_y`, `position_w` | Posição e orientação reportadas pelo robô. |
| `velocity_x`, `velocity_y`, `velocity_w` | Velocidades lineares e angulares reportadas. |
| `dribbler_speed` | Velocidade do dribbler. |
| `capacitor_charge` | Nível de carga do capacitor. |
| `dribbler_ball_contact` | Indica se o IR está ativo. |
| `battery` | Nível da bateria. |
| `count` | Contador de pacotes (para detecção de perdas). |

---

#### 🟦 `raw_robots`

Este dataframe contém os **dados brutos de detecção dos robôs** (tanto aliados quanto adversários) obtidos diretamente das mensagens de visão (*raw frames*) do sistema **SSL-Vision**.

Cada linha representa a detecção de **um robô em um determinado instante (`timestamp`)**.

| Coluna | Descrição |
|:--------|:-----------|
| `timestamp` | Momento exato em que o frame foi publicado pelo sistema de visão. |
| `team` | Nome do time ao qual o robô pertence — pode ser `"robots_blue"` (time azul) ou `"robots_yellow"` (time amarelo). |
| `robot_id` | Identificador único do robô dentro do time (geralmente de 0 a 11). |
| `position_x`, `position_y`, `position_w` | Posição e orientação do robô no campo. |

---

#### 🟠 `raw_ball`

Este dataframe contém os **dados brutos de detecção da bola**, também obtidos a partir dos *raw frames* do **SSL-Vision**.

Cada linha representa a posição detectada da **bola** em um determinado instante (`timestamp`).

| Coluna | Descrição |
|:--------|:-----------|
| `timestamp` | Momento em que a bola foi detectada no frame, em segundos. |
| `position_x`, `position_y` | Posição da bola no campo, em milímetros. |

### Defining important functions

In [15]:
def processed_frame_extract_all_robots_data(filtered_data: list) -> pd.DataFrame:
    robot_data_list = []

    for data in filtered_data:
        if 'processed_frame' not in data:
            continue

        process_frame = data['processed_frame']
        timestamp = process_frame.get('publish_timestamp', 0)

        if timestamp == 0:
            continue

        # Percorre tanto aliados quanto inimigos, se existirem
        for team_name in ['allies', 'enemies']:
            if team_name not in process_frame:
                continue

            for robot in process_frame[team_name]:
                data_line = {
                    'timestamp': timestamp,
                    'team': team_name,
                    'robot_id': robot.get('id')
                }

                # Posição
                if 'position' in robot:
                    data_line['position_x'] = robot['position'].get('x')
                    data_line['position_y'] = robot['position'].get('y')
                    data_line['position_w'] = robot['position'].get('omega')

                # Velocidade
                if 'velocity' in robot:
                    data_line['velocity_x'] = robot['velocity'].get('x')
                    data_line['velocity_y'] = robot['velocity'].get('y')

                robot_data_list.append(data_line)

    return pd.DataFrame(robot_data_list)

In [16]:
def raw_frame_extract_all_robots_data(raw_data: list) -> pd.DataFrame:
    robot_data_list = []

    for data in raw_data:
        if 'raw_frame' not in data or not data['raw_frame']:
            continue

        for raw_frame in data['raw_frame']:
            timestamp = raw_frame.get('publish_timestamp', 0)
            if timestamp == 0:
                continue

            # Verifica se há dados de detecção válidos
            packet = raw_frame.get('packet', {})
            detection = packet.get('detection', {})

            # Itera sobre os dois times, se existirem
            for team_name in ['robots_blue', 'robots_yellow']:
                if team_name not in detection or not detection[team_name]:
                    continue

                for robot in detection[team_name]:
                    data_line = {
                        'timestamp': timestamp,
                        'team': team_name,
                        'robot_id': robot.get('robot_id')
                    }

                    # Posição e orientação
                    data_line['position_x'] = robot.get('x')
                    data_line['position_y'] = robot.get('y')
                    data_line['position_w'] = robot.get('orientation')

                    robot_data_list.append(data_line)

    return pd.DataFrame(robot_data_list)

In [17]:
def robots_command_extract_all_robots_data(filtered_data: list) -> pd.DataFrame:
    robot_command_data_list = []

    for data in filtered_data:
        if 'robots_command' not in data:
            continue

        navigation = data['robots_command'].get('navigation', [])
        if not navigation:
            continue

        for robot in navigation:
            timestamp = robot.get('publish_timestamp', 0)
            if timestamp == 0:
                continue

            data_line = {'timestamp': timestamp, 'robot_id': robot.get('id')}

            # Movimento
            move = robot.get('move', {})
            data_line['move_x'] = move.get('x')
            data_line['move_y'] = move.get('y')
            data_line['move_w'] = move.get('omega')

            # Atuação
            actuation = robot.get('actuation', {})
            data_line['actuation_kick_strength'] = actuation.get('kick_strength')
            data_line['actuation_front'] = actuation.get('front')
            data_line['actuation_chip'] = actuation.get('chip')
            data_line['actuation_charge'] = actuation.get('charge')
            data_line['actuation_dribbler'] = actuation.get('dribbler')
            data_line['actuation_dribbler_velocity'] = actuation.get('dribbler_velocity')

            robot_command_data_list.append(data_line)

    return pd.DataFrame(robot_command_data_list)

In [22]:
def telemetry_extract_all_robots_data(filtered_data: list) -> pd.DataFrame:
    telemetry_data_list = []

    for data in filtered_data:
        if 'telemetry' not in data:
            continue

        telemetries = data['telemetry'].get('telemetries', [])
        if not telemetries:
            continue

        for robot in telemetries:
            timestamp = robot.get('receive_timestamp', 0)
            if timestamp == 0:
                continue

            data_line = {'timestamp': timestamp, 'robot_id': robot.get('id')}

            # Rodas
            for i in range(1, 5):
                data_line[f'wheel{i}'] = robot.get(f'wheel{i}')

            # Posição
            position = robot.get('position', {})
            data_line['position_x'] = position.get('x')
            data_line['position_y'] = position.get('y')
            data_line['position_w'] = position.get('omega')

            # Velocidade
            velocity = robot.get('velocity', {})
            data_line['velocity_x'] = velocity.get('x')
            data_line['velocity_y'] = velocity.get('y')
            data_line['velocity_w'] = velocity.get('omega')

            # Outros parâmetros
            data_line['dribbler_speed'] = robot.get('dribbler_speed')
            data_line['capacitor_charge'] = robot.get('capacitor_charge')
            data_line['dribbler_ball_contact'] = robot.get('dribbler_ball_contact')
            data_line['battery'] = robot.get('battery')
            data_line['count'] = robot.get('count')

            telemetry_data_list.append(data_line)

    return pd.DataFrame(telemetry_data_list)

In [19]:
def extract_kick_events(ball_df: pd.DataFrame, other_dfs: dict, buffer_size=20, pre_frames=100, post_frames=100):
    """
    Extracts kick events from the ball dataframe and slices corresponding rows
    from other dataframes based on matching timestamp ranges.

    Parameters:
        ball_df (pd.DataFrame): Ball dataframe with 'timestamp', 'acceleration_norm', 'velocity_norm'
        other_dfs (dict): Dictionary of other DataFrames, e.g. {'robots': df_robots, 'telemetry': df_telemetry}
        buffer_size (int): Number of samples to use for rolling analysis
        pre_frames (int): Frames to include before the kick start
        post_frames (int): Frames to include after the kick end

    Returns:
        dict: A dictionary with keys 'ball' and one key per other dataframe,
              each containing a list of DataFrame slices corresponding to kick events.
    """

    buffer = []
    sectorIdx = []
    startKick = None
    HasKickEnable = False

    # --- Detect kicks in ball dataframe ---
    for index, row in ball_df.iterrows():
        buffer.append([row['timestamp'], row['acceleration_norm'], row['velocity_norm']])

        if len(buffer) < buffer_size:
            continue

        accLine = [b[1] for b in buffer]
        velLine = [b[2] for b in buffer]

        if (np.average(velLine) > 500 and not HasKickEnable and np.average(accLine) > 1000):
            HasKickEnable = True
            startKick = index
        elif (np.average(velLine) < 10 and HasKickEnable):
            HasKickEnable = False
            sectorIdx.append([startKick, index])

        while len(buffer) > buffer_size:
            buffer.pop(0)

    # --- Collect dataframes for each kick ---
    kicks_data = {'ball': []}
    for name in other_dfs.keys():
        kicks_data[name] = []

    for s, e in sectorIdx:
        # Get timestamp bounds from ball dataframe
        s_idx = int(s - pre_frames)
        e_idx = int(e + post_frames)
        
        # Ball segment
        kicks_data['ball'].append(ball_df.loc[s_idx:e_idx])

        start_ts = kicks_data['ball'][-1].iloc[0]['timestamp']
        end_ts = kicks_data['ball'][-1].iloc[-1]['timestamp']

        # Match data from other dataframes by timestamp range
        for name, df in other_dfs.items():
            mask = (df['timestamp'] >= start_ts) & (df['timestamp'] <= end_ts)
            kicks_data[name].append(df[mask])

    return kicks_data

### Extract Dataframes

In [20]:
df_processed_robots = processed_frame_extract_all_robots_data(data_list)
df_raw_robots = raw_frame_extract_all_robots_data(data_list)
df_commands = robots_command_extract_all_robots_data(data_list)
df_telemetry = telemetry_extract_all_robots_data(data_list)
raw_ball = raw_frame_extract_ball_data_frame(data_list)
ball = processed_frame_extract_ball_data_frame(data_list)

### Process timestamps

In [27]:
print(ball.shape)
print(ball.columns)      # vai mostrar RangeIndex? (ex.: RangeIndex(start=0, stop=N, step=1))
print(ball.head(3))
print(ball.index.name, df_telemetry.index.dtype)

(277252, 7)
Index(['timestamp', 'position_x', 'position_y', 'velocity_x', 'velocity_y',
       'acceleration_x', 'acceleration_y'],
      dtype='object')
             timestamp  position_x  position_y  velocity_x  velocity_y  \
0  1761268186634745432   -7.226245   -0.433875   -0.089194    2.713994   
1  1761268186651723664   -7.725473   -0.638854   -0.707642    2.486564   
2                    0    0.000000    0.000000    0.000000    0.000000   

   acceleration_x  acceleration_y  
0       14.373293        7.744687  
1      -10.966610       -2.462649  
2        0.000000        0.000000  
None int64


In [28]:
# CONVERT TIMESTAMPS TO FLOAT
df_processed_robots['timestamp'] = df_processed_robots['timestamp'].astype(float)
df_raw_robots['timestamp'] = df_raw_robots['timestamp'].astype(float)
df_commands['timestamp'] = df_commands['timestamp'].astype(float)
#df_telemetry['timestamp'] = df_telemetry['timestamp'].astype(float)
raw_ball['timestamp'] = raw_ball['timestamp'].astype(float)
ball['timestamp'] = ball['timestamp'].astype(float)

# FILTER DATA WITH NEGATIVE TIMESTAMP
df_processed_robots = df_processed_robots[df_processed_robots['timestamp'] > 1]
df_raw_robots = df_raw_robots[df_raw_robots['timestamp'] > 1]
df_commands = df_commands[df_commands['timestamp'] > 1]
#df_telemetry = df_telemetry[df_telemetry['timestamp'] > 1]
raw_ball = raw_ball[raw_ball['timestamp'] > 1]
ball = ball[ball['timestamp'] > 1]

# GET TIME REFERENCE
timeref = min(ball['timestamp'].values[0], raw_ball['timestamp'].values[0])

# CONVERT FROM NANOSECONDS TO MILLISECONDS
df_processed_robots['timestamp'] = (df_processed_robots['timestamp'] - timeref).apply(lambda x: float(x)%1e13/1e9)
df_raw_robots['timestamp'] = (df_raw_robots['timestamp'] - timeref).apply(lambda x: float(x)%1e13/1e9)
df_commands['timestamp'] = (df_commands['timestamp'] - timeref).apply(lambda x: float(x)%1e13/1e9)
#df_telemetry['timestamp'] = (df_telemetry['timestamp'] - timeref).apply(lambda x: float(x)%1e13/1e9)
raw_ball['timestamp'] = (raw_ball['timestamp'] - timeref).apply(lambda x: float(x)%1e13/1e9)
ball['timestamp'] = (ball['timestamp'] - timeref).apply(lambda x: float(x)%1e13/1e9)

### Create columns for velocity and acceleration norms

In [29]:
# CREATE COLUMNS FOR VELOCITY AND ACCELERATION NORMS
df_processed_robots['velocity_norm'] = gel_2d_length_in_column(df_processed_robots, 'velocity')
ball['velocity_norm'] = gel_2d_length_in_column(ball, 'velocity')
ball['acceleration_norm'] = gel_2d_length_in_column(ball, 'acceleration')

### Printing head of Dataframes

In [30]:
df_processed_robots.head()

,timestamp,team,robot_id,position_x,position_y,position_w,velocity_x,velocity_y,velocity_norm
0,0.000298,allies,0,5591.743607,1.154007,3.135174,-0.763675,0.586366,0.962821
1,0.000298,allies,1,1708.493708,-840.202168,3.101607,-4.571530,-0.550978,4.604613
2,0.000298,allies,2,837.103112,-0.456033,-3.084250,0.441652,3.603632,3.630595
3,0.000298,allies,3,1992.443342,145.766129,-3.090637,1.335969,-1.788187,2.232135
4,0.000298,allies,4,3955.475036,-11.715136,-3.106912,-0.267227,1.487415,1.511229


In [31]:
df_raw_robots.head()

,timestamp,team,robot_id,position_x,position_y,position_w
0,0.0,robots_blue,2,-3449.196533,-491.030060,0.139911
1,0.0,robots_blue,7,-4024.627441,91.213127,-0.025321
2,0.0,robots_blue,10,-3773.889648,-331.019531,0.099557
3,0.0,robots_blue,12,-4023.330566,-95.527031,0.018154
4,0.0,robots_blue,6,-632.789673,-2.312718,0.002433


In [32]:
df_commands.head()

,timestamp,robot_id,move_x,move_y,move_w,actuation_kick_strength,actuation_front,actuation_chip,actuation_charge,actuation_dribbler,actuation_dribbler_velocity
0,9999.997431,5,-0.585422,-0.151575,-0.109092,0.0,False,False,False,False,0.0
1,0.000112,2,-0.749209,0.114882,-0.143360,0.0,False,False,False,False,0.0
2,0.002791,3,0.000000,0.000000,0.000000,0.0,False,False,False,False,0.0
3,0.002821,0,0.000000,-0.000000,0.015568,6.5,False,True,False,False,0.0
4,0.002855,3,0.000000,0.000000,0.000000,0.0,False,False,False,False,0.0


In [33]:
#df_telemetry.head()

In [34]:
ball.head()

,timestamp,position_x,position_y,velocity_x,velocity_y,acceleration_x,acceleration_y,velocity_norm,acceleration_norm
0,0.000298,-7.226245,-0.433875,-0.089194,2.713994,14.373293,7.744687,2.715459,16.327024
1,0.017277,-7.725473,-0.638854,-0.707642,2.486564,-10.966610,-2.462649,2.585297,11.239714
3,0.032729,-7.803468,-0.508317,-2.423138,1.233601,-34.447629,-17.460829,2.719076,38.620197
4,0.046356,-7.242591,-1.022733,-1.211956,0.461296,3.555078,-37.144850,1.296777,37.314588
5,0.057252,-7.070012,-1.383479,0.428070,-1.604145,18.145693,-52.351221,1.660279,55.406828


In [35]:
raw_ball.head()

,timestamp,position_x,position_y
0,0.000000,-7.851546,1.061366
1,0.017108,-7.217216,-2.283789
2,0.042477,-7.560882,-0.308831
3,0.046134,-5.806601,-3.558045
4,0.056912,-8.311053,0.373733


### Process Dataframe and save data

In [36]:
other_dfs = {
    'processed_robots': df_processed_robots,
    'raw_robots': df_raw_robots,
#    'telemetry': df_telemetry,
    'commands': df_commands,
    'raw_ball': raw_ball
}

kicks_data = extract_kick_events(ball, other_dfs)

In [38]:
# --- Save plots and CSVs for each kick event ---

count = 0
for kick_idx in range(len(kicks_data['ball'])):
    # --- Plot ball velocity for this kick ---
    kick_ball = kicks_data['ball'][kick_idx]
    plt.plot(kick_ball.index, kick_ball['velocity_norm'])
    label_base = f"{OUTPUT_DIR}/kickEvent{count}"

    plt.savefig(label_base + '.png', dpi=300, bbox_inches='tight')
    plt.clf()

    # --- Save CSV for each dataframe (ball + others) ---
    for df_name, df_list in kicks_data.items():
        if len(df_list) <= kick_idx:
            continue  # skip if this df has fewer events

        df_event = df_list[kick_idx]
        if df_event.empty:
            continue  # skip empty dataframes

        df_event.to_csv(f"{label_base}_{df_name}.csv", index=False)

    count += 1
